# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.04157


extracting tarball to tmp_2303.04157...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.04157/h4dev.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:359: LatexWarning: Latex injecting: 'app_sel' from 'tmp_2303.04157/app_sel.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:359: LatexWarning: Latex injecting: 'app_agebias' from 'tmp_2303.04157/app_agebias.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_de

...

✔ → 65319:\section{Discussion}\label{sec:discussion}
  ↳ 90925:\section{Conclusions and outlook}\label{sec:conclusion}
✔ → 90925:\section{Conclusions and outlook}\label{sec:conclusion}
  ↳ 95364:\begin{appendix}
✔ → 95364:\begin{appendix}
  ↳ 95381:\section{Cloud retrieval validation}\label{sec:validation}


✔ → 95381:\section{Cloud retrieval validation}\label{sec:validation}
  ↳ 103682:\section{Equilibrium temperature and Bond albedo}\label{sec:albedo_calc}
✘ → 103682:\section{Equilibrium temperature and Bond albedo}\label{sec:albedo_calc}
  ↳ 106102:\section{Additional plots for retrieval results}\label{app:corners}


✔ → 106102:\section{Additional plots for retrieval results}\label{app:corners}
  ↳ 114276:\section{Supplementary results from retrievals on higher quality input spectra}\label{app:extended_retrievals}


✔ → 114276:\section{Supplementary results from retrievals on higher quality input spectra}\label{app:extended_retrievals}
  ↳ 121913:\section{Cloudy vs. icy scenario - comparison of absolute fluxes}\label{app:abs_flux_comp}
✔ → 121913:\section{Cloudy vs. icy scenario - comparison of absolute fluxes}\label{app:abs_flux_comp}
  ↳ 124034:end


Found 97 bibliographic references in tmp_2303.04727/aanda.bbl.
syntax error in line 95: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04727-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04727) | **Large Interferometer For Exoplanets (LIFE): X. Assessing the Impact of  Clouds on Atmospheric Retrievals at Mid-Infrared Wavelengths with a  Venus-Twin Exoplanet**  |
|| B. S. Konrad, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *Re-submitted to A&A after a first iteration with referee; 15 pages (main text incl. 8 figures and 6 tables) + appendix; comments are welcome. This paper is part of a series on the LIFE telescope. Related series papers: arXiv:2101.07500, arXiv:2203.00471, arXiv:2112.02054, arXiv:2204.10041*|
|**Abstract**| The Large Interferometer For Exoplanets (LIFE) initiative aims to develop a space based mid-infrared (MIR) nulling interferometer to measure the thermal emission spectra of temperate terrestrial exoplanets. We investigate how well LIFE could characterize a cloudy Venus-twin exoplanet to: (1) test our retrieval routine on a realistic non-Earth-like MIR spectrum of a known planet, (2) investigate how clouds impact retrievals, (3) refine the LIFE requirements derived in previous Earth-centered studies. We run retrievals for simulated LIFE observations of a Venus-twin exoplanet orbiting a Sun-like star located 10 pc from the observer. By assuming different models (cloudy and cloud-free) we analyze the performance as a function of the quality of the LIFE observation. This allows us to determine how well atmosphere and clouds are characterizable depending on the quality of the spectrum. Our study shows that the current minimal resolution ($R=50$) and signal-to-noise ($S/N=10$ at $11.2\mu$m) requirements for LIFE suffice to characterize the structure and composition of a Venus-like atmosphere above the cloud deck if an adequate model is chosen. However, we cannot infer cloud properties. The accuracy of the retrieved planet radius ($R_{pl}$), equilibrium temperature ($T_{eq}$), and Bond albedo ($A_B$) depend on the choice of model. Generally, a cloud-free model performs best and thus the presence of clouds cannot be inferred. This model dependence of retrieval results emphasizes the importance of developing a community-wide best-practice for atmospheric retrieval studies. If we consider higher quality spectra (especially $S/N=20$), we can infer the presence of clouds and pose first constraints on their structure. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04180-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04180) | **The Atacama Cosmology Telescope: Map-Based Noise Simulations for DR6**  |
|| Zachary Atkins, et al. -- incl., <mark>Frank J. Qu</mark>, <mark>Zack Li</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *41 pages (+10 appendix), 22 figures (+5 appendix), submitted to JCAP*|
|**Abstract**| The increasing statistical power of cosmic microwave background (CMB) datasets requires a commensurate effort in understanding their noise properties. The noise in maps from ground-based instruments is dominated by large-scale correlations, which poses a modeling challenge. This paper develops novel models of the complex noise covariance structure in the Atacama Cosmology Telescope Data Release 6 (ACT DR6) maps. We first enumerate the noise properties that arise from the combination of the atmosphere and the ACT scan strategy. We then prescribe a class of Gaussian, map-based noise models, including a new wavelet-based approach that uses directional wavelet kernels for modeling correlated instrumental noise. The models are empirical, whose only inputs are a small number of independent realizations of the same region of sky. We evaluate the performance of these models against the ACT DR6 data by drawing ensembles of noise realizations. Applying these simulations to the ACT DR6 power spectrum pipeline reveals a $\sim 20\%$ excess in the covariance matrix diagonal when compared to an analytic expression that assumes noise properties are uniquely described by their power spectrum. Along with our public code, $\mathtt{mnms}$, this work establishes a necessary element in the science pipelines of both ACT DR6 and future ground-based CMB experiments such as the Simons Observatory (SO). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04184-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04184) | **A candidate magnetic helium core white dwarf in the globular cluster NGC  6397**  |
|| Manuel Pichardo Marcano, et al. -- incl., <mark>Leandro G. Althaus</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| We report a peculiar variable blue star in the globular cluster NGC 6397, using Hubble Space Telescope optical imaging. Its position in the colour-magnitude diagrams, and its spectrum, are consistent with this star being a helium core white dwarf (He WD) in a binary system. The optical light curve shows a periodicity at 18.5 hours. We argue that this periodicity is due to the rotation of the WD and possibly due to magnetic spots on the surface of the WD. This would make this object the first candidate magnetic He WD in any globular cluster (GC), and the first candidate magnetic WD in a detached binary system in any GC and one of the few He WDs with a known rotation period and of magnetic nature. Another possibility is that this system is a He WD in a binary system with another WD or another degenerate object, which would make this object one of the few candidate non-accreting double degenerate binaries in any GC. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04412-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04412) | **Multiwavelength observations of the extraordinary accretion event  AT2021lwx**  |
|| P. Wiseman, et al. -- incl., <mark>T. E. Müller-Bravo</mark>, <mark>K. Smith</mark>, <mark>M. Smith</mark>, <mark>M. Huber</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *11 pages, 5 figures, submitted to MNRAS*|
|**Abstract**| We present observations from X-ray to mid-infrared wavelengths of the most energetic non-quasar transient ever observed, AT2021lwx. Our data show a single optical brightening by a factor $>100$ to a luminosity of $7\times10^{45}$ erg s$^{-1}$, and a total radiated energy of $1.5\times10^{53}$ erg, both greater than any known optical transient. The decline is smooth and exponential and the ultra-violet - optical spectral energy distribution resembles a black body with temperature $1.2\times10^4$ K. Tentative X-ray detections indicate a secondary mode of emission, while a delayed mid-infrared flare points to the presence of dust surrounding the transient. The spectra are similar to recently discovered optical flares in known active galactic nuclei but lack some characteristic features. The lack of emission for the previous seven years is inconsistent with the short-term, stochastic variability observed in quasars, while the extreme luminosity and long timescale of the transient disfavour the disruption of a single solar-mass star. The luminosity could be generated by the disruption of a much more massive star, but the likelihood of such an event occurring is small. A plausible scenario is the accretion of a giant molecular cloud by a dormant black hole of $10^8 - 10^9$ solar masses. AT2021lwx thus represents an extreme extension of the known scenarios of black hole accretion. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04421-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04421) | **Monitoring the Size and Flux Density of Sgr A* during the Active State  in 2019 with East Asian VLBI Network**  |
|| Xiaopeng Cheng, et al. -- incl., <mark>Guang-Yao Zhao</mark>, <mark>Sang-Sung Lee</mark>, <mark>Jee-Won Lee</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *15 pages, 7 figures, 3 tables, Published in the Galaxies Special Issue "Challenges in Understanding Black Hole Powered Jets with VLBI"*|
|**Abstract**| In this work, we studied the Galactic Center supermassive black hole (SMBH), Sagittarius A* (Sgr A*), with the KVN and VERA Array (KaVA)/East Asian VLBI Network (EAVN) monitoring observations. Especially on 13 May 2019, Sgr A* experienced an unprecedented bright near infra-red (NIR) flare; so, we find a possible counterpart at 43 GHz (7 mm). As a result, a large temporal variation of the flux density at the level 15.4%, with the highest flux density of 2.04 Jy, is found on 11 May 2019. Interestingly, the intrinsic sizes are also variable, and the area and major-axis size show a marginal correlation with flux density with >2{\sigma}. Thus, we interpret that the emission region at 43 GHz follows the larger-when-brighter relation in 2019. The possible origins are discussed with an emergence of a weak jet/outflow component and the position angle change of the rotation axis of the accretion disk in time. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04726-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04726) | **PEARLS: Low Stellar Density Galaxies in the El Gordo Cluster Observed  with JWST**  |
|| Timothy Carleton, et al. -- incl., <mark>Jiashuo Zhang</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *Submitted to ApJ. Comments welcome!*|
|**Abstract**| A full understanding of how unusually large "Ultra Diffuse Galaxies" (UDGs) fit into our conventional theory of dwarf galaxy formation remains elusive, despite the large number of objects identified locally. A natural extension of UDG research is the study of similar galaxies at higher redshift to establish how their properties may evolve over time. However, this has been a challenging task given how severely cosmological surface brightness dimming inhibits our ability to detect low-surface brightness galaxies at high-$z$. Here, we present a sample of low stellar surface density galaxies (LDGs) at moderate redshift, likely the progenitors of local UDGs, identified in deep near-IR {\it JWST} observations of the El Gordo cluster at $z=0.87$. By stacking 8 NIRCAM filters, reach an apparent surface brightness sensitivity of 24.59~mag~arcsec$^{-2}$, faint enough to be complete to the bright end of the LDG population. Our analysis identifies significant differences between this population and UDGs observed locally, including differences in their color and size distributions, which suggest that the UDG progenitors at high-$z$ are bluer and more extended than UDGs at $z=0$. This suggests that multiple mechanisms are responsible for UDG formation and that prolonged transformation of cluster dwarfs is not a primary UDG formation mechanism at high-$z$. Furthermore, we find a slight overabundance of LDGs in El Gordo, and, in contrast to findings in local clusters, our analysis does not show a deficit of LDGs in the center of El Gordo, implying that tidal destruction of LDGs is significant between $z=0.87$ and $z=0$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.04157-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.04157) | **Different higher-order kinematics between star-forming and quiescent  galaxies based on the SAMI, MAGPI and LEGA-C surveys**  |
|| Francesco D'Eugenio, et al. -- incl., <mark>Anna de Graaff</mark>, <mark>Gauri Sharma</mark> |
|*Appeared on*| *2023-03-09*|
|*Comments*| *26 pages, 15 figures Accepted for publication in MNRAS*|
|**Abstract**| We present the first statistical study of spatially integrated non-Gaussian stellar kinematics spanning 7 Gyr in cosmic time. We use deep, rest-frame optical spectroscopy of massive galaxies (stellar mass $M_\star > 10^{10.5} {\rm M}_\odot$) at redshifts z = 0.05, 0.3 and 0.8 from the SAMI, MAGPI and LEGA-C surveys, to measure the excess kurtosis $h_4$ of the stellar velocity distribution, the latter parametrised as a Gauss-Hermite series. We find that at all redshifts where we have large enough samples, $h_4$ anti-correlates with the ratio between rotation and dispersion, highlighting the physical connection between these two kinematic observables. In addition, and independently from the anti-correlation with rotation-to-dispersion ratio, we also find a correlation between $h_4$ and $M_\star$, potentially connected to the assembly history of galaxies. In contrast, after controlling for mass, we find no evidence of independent correlation between $h_4$ and aperture velocity dispersion or galaxy size. These results hold for both star-forming and quiescent galaxies. For quiescent galaxies, $h_4$ also correlates with projected shape, even after controlling for the rotation-to-dispersion ratio. At any given redshift, star-forming galaxies have lower $h_4$ compared to quiescent galaxies, highlighting the link between kinematic structure and star-forming activity. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure f.tmods.models.b</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.04727.md
    + _build/html/tmp_2303.04727/./Figures/Validation/val_corner.png
    + _build/html/tmp_2303.04727/./Figures/Validation/val_pt_structure_residual.png
    + _build/html/tmp_2303.04727/./Figures/Validation/val_spectrum_residual.png
    + _build/html/tmp_2303.04727/./Figures/R50_SN10/Corner_O_H2SO4.png
    + _build/html/tmp_2303.04727/./Figures/R50_SN10/PT_O_H2SO4.png
    + _build/html/tmp_2303.04727/./Figures/R50_SN10/Spec_O_H2SO4.png
    + _build/html/tmp_2303.04727/./Figures/R50_SN10/EmCont_O_H2SO4.png
    + _build/html/tmp_2303.04727/./Figures/Venus_Cloud_vs_Icy.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\R}[0]{\ensuremath{R}}$
$\newcommand{\Rv}[1]{\ensuremath{R = #1}}$
$\newcommand{\SN}[0]{\ensuremath{S/N}}$
$\newcommand{\SNv}[1]{\ensuremath{S/N = #1}}$
$\newcommand{\mic}[1]{\ensuremath{#1}~\textmu m}$
$\newcommand{\pt}[0]{\textit{P}-\textit{T}}$
$\newcommand{\Rpl}[0]{\ensuremath{R_{\text{pl}}}}$
$\newcommand{\Mpl}[0]{\ensuremath{M_{\text{pl}}}}$
$\newcommand{\Pct}[0]{\ensuremath{P\mathrm{^{cloud}_{top}}}}$
$\newcommand{\Tct}[0]{\ensuremath{T\mathrm{^{cloud}_{top}}}}$
$\newcommand{\Teq}[0]{\ensuremath{T_\mathrm{eq}}}$
$\newcommand{\Pcs}[0]{\ensuremath{P\mathrm{^{cloud}_{span}}}}$
$\newcommand{\Rcm}[0]{\ensuremath{\bar{R}\mathrm{^{cloud}}}}$
$\newcommand{\Sc}[0]{\ensuremath{\sigma\mathrm{^{cloud}}}}$
$\newcommand{\Ab}[0]{\ensuremath{A_\mathrm{B}}}$
$\newcommand{\Ps}[0]{\ensuremath{P_0}}$
$\newcommand{\Ts}[0]{\ensuremath{T_0}}$
$\newcommand{\MSaOp}[0]{\ensuremath{\mathcal{M}^{\ce{H2SO4}}_{\mathrm{Op.}}}}$
$\newcommand{\MSaTr}[0]{\ensuremath{\mathcal{M}^{\ce{H2SO4}}_{\mathrm{Tr.}}}}$
$\newcommand{\MWaOp}[0]{\ensuremath{\mathcal{M}^{\ce{H2O}}_{\mathrm{Op.}}}}$
$\newcommand{\MCf}[0]{\ensuremath{\mathcal{M}^\mathrm{CF}}}$
$\newcommand{\lgrt}[1]{\ensuremath{\log_{10}(#1)}}$
$\newcommand{\lgrtdaj}[1]{\ensuremath{\log_{10}\left(#1\right)}}$
$\newcommand{\life}[0]{LIFE}$
$\newcommand{\lifesim}[0]{LIFE\textsc{sim}}$
$\newcommand{\pI}[0]{Paper~I}$
$\newcommand{\pII}[0]{Paper~II}$
$\newcommand{\pIII}[0]{Paper~III}$
$\newcommand{\pV}[0]{Paper~V}$
$\newcommand{\pIIIaV}[0]{Papers~III and~V}$
$\newcommandinecolor{tab:OpH2SO4}{RGB}{0, 158, 115}$
$\newcommandinecolor{tab:OpH2SO4t}{RGB}{191, 230, 219}$
$\newcommandinecolor{tab:TrH2SO4}{RGB}{0, 114, 178}$
$\newcommandinecolor{tab:TrH2SO4t}{RGB}{191, 219, 235}$
$\newcommandinecolor{tab:OpH2O}{RGB}{213, 94, 0}$
$\newcommandinecolor{tab:OpH2Ot}{RGB}{244, 214, 191}$
$\newcommandinecolor{tab:CF}{RGB}{230, 159, 0}$
$\newcommandinecolor{tab:CFt}{RGB}{248, 230, 191}$
$   \title{Large Interferometer For Exoplanets (LIFE):}$
$   \subtitle{X. Assessing the Impact of Clouds on Atmospheric Retrievals at Mid-Infrared Wavelengths with a Venus-Twin Exoplanet}$
$   \titlerunning{LIFE: X. Assessing the Impact of Clouds on Atmospheric Retrievals with a Venus-Twin Exoplanet}$
$   \authorrunning{Konrad et al.}$
$   \author{$
$    B.S. Konrad \inst{1,2}$
$    \and$
$    E. Alei \inst{1,2}$
$    \and$
$    S.P. Quanz \inst{1,2,3}$
$    \and$
$    P. Mollière \inst{4}$
$    \and$
$    D. Angerhausen \inst{1,2}$
$    \and$
$    J.J. Fortney \inst{5}$
$    \and$
$    K. Hakim \inst{2,6}$
$    \and$
$    S. Jordan \inst{7}$
$    \and$
$    D. Kitzmann \inst{2,6}$
$    \and$
$    S. Rugheimer \inst{8}$
$    \and$
$    O. Shorttle \inst{7,9}$
$    \and$
$    R. Wordsworth \inst{10}$
$   \and$
$   the \textit{LIFE} Collaboration\thanks{Webpage: \url{www.life-space-mission.com}}$
$   }$
$  \institute{$
$    ETH Zurich, Institute for Particle Physics \& Astrophysics, Wolfgang-Pauli-Str. 27, 8093 Zurich, Switzerland\e-mail:\texttt{konradb@ethz.ch}; \texttt{sascha.quanz@phys.ethz.ch}$
$    \and$
$    National Center of Competence in Research PlanetS (www.nccr-planets.ch)$
$    \and$
$    ETH Centre for Origin and Prevalence of Life, Wolfgang-Pauli-Str. 27, 8093 Zurich, Switzerland$
$    \and$
$    Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany$
$    \and$
$    Department of Astronomy and Astrophysics, University of California, Santa Cruz, CA, USA 95064$
$    \and$
$    University of Bern, Center for Space and Habitability, Gesellschaftsstrasse 6, 3012 Bern, Switzerland$
$    \and$
$    Institute of Astronomy, University of Cambridge, CB3 0HA, UK$
$    \and$
$    Department of Physics and Astronomy, York University, 4700 Keele Street, North York, Ontario 3MJ 1P3, Canada$
$    \and$
$    Department of Earth Sciences, University of Cambridge, CB2 3EQ, UK$
$    \and$
$    School of Engineering and Applied Sciences, Harvard University, Cambridge, MA 02138, USA$
$}$
$   \date{Received -; accepted -}$
$  \abstract{Terrestrial exoplanets in the habitable zone are likely a common occurrence. The long-term goal is to characterize the atmospheres of dozens of such objects. The Large Interferometer For Exoplanets (\life ) initiative aims to develop a spacebased mid-infrared (MIR) nulling interferometer to measure the thermal emission spectra of such exoplanets.}{We investigate how well \life  could characterize a cloudy Venus-twin exoplanet. This allows us to: (1) test our atmospheric retrieval routine on a realistic non-Earth-like MIR emission spectrum of a known planet, (2) investigate how clouds impact retrievals, (3) further refine the \life  requirements derived in previous Earth-centered studies.}{We run Bayesian atmospheric retrievals for simulated \life  observations of a Venus-twin exoplanet. Our exoplanet orbits a Sun-like star located 10~pc from the observer. The \lifesim  noise model accounts for all major astrophysical noise sources. We run retrievals using different models (cloudy and cloud-free) and analyze the performance as a function of the quality of the \life  observation. This allows us to determine how well atmosphere and clouds are characterizable depending on the quality of the spectrum.}{At the current minimal resolution (\Rv{50}) and signal-to-noise (\SNv{10} at \mic{11.2}) requirements for \life , all tested models suggest a \ce{CO2}-rich atmosphere (\geq30\% in mass fraction). Further, we successfully constrain the atmospheric pressure-temperature (\pt ) structure above the cloud deck (\pt  uncertainty \leq\pm15~K). However, we struggle to infer the main cloud properties. Further, the retrieved planetary radius (\Rpl ), equilibrium temperature (\Teq ), and Bond albedo (\Ab ) depend on the model. Generally, a cloud-free model performs best at the current minimal quality and accurately estimates \Rpl , \Teq , and \Ab . If we consider higher quality spectra (especially \SNv{20}), we can infer the presence of clouds and pose first constraints on their structure.}{Our study shows that the minimal \R  and \SN  requirements for \life  suffice to characterize the structure and composition of a Venus-like atmosphere above the cloud deck if an adequate model is chosen. Crucially, the cloud-free model is preferred by the retreival for low spectral qualities. We thus find no direct evidence for clouds at the minimal \R  and \SN  requirements and cannot infer the thickness of the atmosphere. Clouds are only constrainable in MIR retrievals of spectra with \SN \geq20. The model dependence of our retrieval results emphasizes the importance of developing a community-wide best-practice for atmospheric retrieval studies.$
$  }$
$   \keywords{   Methods: statistical --$
$                Planets and satellites: terrestrial planets --$
$                Planets and satellites: atmospheres$
$                }$
$   \maketitle$
$\n\end{document}}$
$\newcommand{\life}[0]{LIFE}$
$\newcommand{\lifesim}[0]{LIFE\textsc{sim}}$
$\newcommand{\pI}[0]{Paper~I}$
$\newcommand{\pII}[0]{Paper~II}$
$\newcommand{\pIII}[0]{Paper~III}$
$\newcommand{\pV}[0]{Paper~V}$
$\newcommand{\pIIIaV}[0]{Papers~III and~V}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\R}[0]{\ensuremath{R}}$
$\newcommand{\Rv}[1]{\ensuremath{R = #1}}$
$\newcommand{\SN}[0]{\ensuremath{S/N}}$
$\newcommand{\SNv}[1]{\ensuremath{S/N = #1}}$
$\newcommand{\mic}[1]{\ensuremath{#1}~\textmu m}$
$\newcommand{\pt}[0]{\textit{P}-\textit{T}}$
$\newcommand{\Rpl}[0]{\ensuremath{R_{\text{pl}}}}$
$\newcommand{\Mpl}[0]{\ensuremath{M_{\text{pl}}}}$
$\newcommand{\Pct}[0]{\ensuremath{P\mathrm{^{cloud}_{top}}}}$
$\newcommand{\Tct}[0]{\ensuremath{T\mathrm{^{cloud}_{top}}}}$
$\newcommand{\Teq}[0]{\ensuremath{T_\mathrm{eq}}}$
$\newcommand{\Pcs}[0]{\ensuremath{P\mathrm{^{cloud}_{span}}}}$
$\newcommand{\Rcm}[0]{\ensuremath{\bar{R}\mathrm{^{cloud}}}}$
$\newcommand{\Sc}[0]{\ensuremath{\sigma\mathrm{^{cloud}}}}$
$\newcommand{\Ab}[0]{\ensuremath{A_\mathrm{B}}}$
$\newcommand{\Ps}[0]{\ensuremath{P_0}}$
$\newcommand{\Ts}[0]{\ensuremath{T_0}}$
$\newcommand{\MSaOp}[0]{\ensuremath{\mathcal{M}^{\ce{H2SO4}}_{\mathrm{Op.}}}}$
$\newcommand{\MSaTr}[0]{\ensuremath{\mathcal{M}^{\ce{H2SO4}}_{\mathrm{Tr.}}}}$
$\newcommand{\MWaOp}[0]{\ensuremath{\mathcal{M}^{\ce{H2O}}_{\mathrm{Op.}}}}$
$\newcommand{\MCf}[0]{\ensuremath{\mathcal{M}^\mathrm{CF}}}$
$\newcommand{\lgrt}[1]{\ensuremath{\log_{10}(#1)}}$
$\newcommand{\lgrtdaj}[1]{\ensuremath{\log_{10}\left(#1\right)}}$
$\newcommand{\life}[0]{LIFE}$
$\newcommand{\lifesim}[0]{LIFE\textsc{sim}}$
$\newcommand{\pI}[0]{Paper~I}$
$\newcommand{\pII}[0]{Paper~II}$
$\newcommand{\pIII}[0]{Paper~III}$
$\newcommand{\pV}[0]{Paper~V}$
$\newcommand{\pIIIaV}[0]{Papers~III and~V}$
$\newcommandinecolor{tab:OpH2SO4}{RGB}{0, 158, 115}$
$\newcommandinecolor{tab:OpH2SO4t}{RGB}{191, 230, 219}$
$\newcommandinecolor{tab:TrH2SO4}{RGB}{0, 114, 178}$
$\newcommandinecolor{tab:TrH2SO4t}{RGB}{191, 219, 235}$
$\newcommandinecolor{tab:OpH2O}{RGB}{213, 94, 0}$
$\newcommandinecolor{tab:OpH2Ot}{RGB}{244, 214, 191}$
$\newcommandinecolor{tab:CF}{RGB}{230, 159, 0}$
$\newcommandinecolor{tab:CFt}{RGB}{248, 230, 191}$
$   \title{Large Interferometer For Exoplanets (LIFE):}$
$   \subtitle{X. Assessing the Impact of Clouds on Atmospheric Retrievals at Mid-Infrared Wavelengths with a Venus-Twin Exoplanet}$
$   \titlerunning{LIFE: X. Assessing the Impact of Clouds on Atmospheric Retrievals with a Venus-Twin Exoplanet}$
$   \authorrunning{Konrad et al.}$
$   \author{$
$    B.S. Konrad \inst{1,2}$
$    \and$
$    E. Alei \inst{1,2}$
$    \and$
$    S.P. Quanz \inst{1,2,3}$
$    \and$
$    P. Mollière \inst{4}$
$    \and$
$    D. Angerhausen \inst{1,2}$
$    \and$
$    J.J. Fortney \inst{5}$
$    \and$
$    K. Hakim \inst{2,6}$
$    \and$
$    S. Jordan \inst{7}$
$    \and$
$    D. Kitzmann \inst{2,6}$
$    \and$
$    S. Rugheimer \inst{8}$
$    \and$
$    O. Shorttle \inst{7,9}$
$    \and$
$    R. Wordsworth \inst{10}$
$   \and$
$   the \textit{LIFE} Collaboration\thanks{Webpage: \url{www.life-space-mission.com}}$
$   }$
$  \institute{$
$    ETH Zurich, Institute for Particle Physics \& Astrophysics, Wolfgang-Pauli-Str. 27, 8093 Zurich, Switzerland\e-mail:\texttt{konradb@ethz.ch}; \texttt{sascha.quanz@phys.ethz.ch}$
$    \and$
$    National Center of Competence in Research PlanetS (www.nccr-planets.ch)$
$    \and$
$    ETH Centre for Origin and Prevalence of Life, Wolfgang-Pauli-Str. 27, 8093 Zurich, Switzerland$
$    \and$
$    Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany$
$    \and$
$    Department of Astronomy and Astrophysics, University of California, Santa Cruz, CA, USA 95064$
$    \and$
$    University of Bern, Center for Space and Habitability, Gesellschaftsstrasse 6, 3012 Bern, Switzerland$
$    \and$
$    Institute of Astronomy, University of Cambridge, CB3 0HA, UK$
$    \and$
$    Department of Physics and Astronomy, York University, 4700 Keele Street, North York, Ontario 3MJ 1P3, Canada$
$    \and$
$    Department of Earth Sciences, University of Cambridge, CB2 3EQ, UK$
$    \and$
$    School of Engineering and Applied Sciences, Harvard University, Cambridge, MA 02138, USA$
$}$
$   \date{Received -; accepted -}$
$  \abstract{Terrestrial exoplanets in the habitable zone are likely a common occurrence. The long-term goal is to characterize the atmospheres of dozens of such objects. The Large Interferometer For Exoplanets (\life ) initiative aims to develop a spacebased mid-infrared (MIR) nulling interferometer to measure the thermal emission spectra of such exoplanets.}{We investigate how well \life  could characterize a cloudy Venus-twin exoplanet. This allows us to: (1) test our atmospheric retrieval routine on a realistic non-Earth-like MIR emission spectrum of a known planet, (2) investigate how clouds impact retrievals, (3) further refine the \life  requirements derived in previous Earth-centered studies.}{We run Bayesian atmospheric retrievals for simulated \life  observations of a Venus-twin exoplanet. Our exoplanet orbits a Sun-like star located 10~pc from the observer. The \lifesim  noise model accounts for all major astrophysical noise sources. We run retrievals using different models (cloudy and cloud-free) and analyze the performance as a function of the quality of the \life  observation. This allows us to determine how well atmosphere and clouds are characterizable depending on the quality of the spectrum.}{At the current minimal resolution (\Rv{50}) and signal-to-noise (\SNv{10} at \mic{11.2}) requirements for \life , all tested models suggest a \ce{CO2}-rich atmosphere (\geq30\% in mass fraction). Further, we successfully constrain the atmospheric pressure-temperature (\pt ) structure above the cloud deck (\pt  uncertainty \leq\pm15~K). However, we struggle to infer the main cloud properties. Further, the retrieved planetary radius (\Rpl ), equilibrium temperature (\Teq ), and Bond albedo (\Ab ) depend on the model. Generally, a cloud-free model performs best at the current minimal quality and accurately estimates \Rpl , \Teq , and \Ab . If we consider higher quality spectra (especially \SNv{20}), we can infer the presence of clouds and pose first constraints on their structure.}{Our study shows that the minimal \R  and \SN  requirements for \life  suffice to characterize the structure and composition of a Venus-like atmosphere above the cloud deck if an adequate model is chosen. Crucially, the cloud-free model is preferred by the retreival for low spectral qualities. We thus find no direct evidence for clouds at the minimal \R  and \SN  requirements and cannot infer the thickness of the atmosphere. Clouds are only constrainable in MIR retrievals of spectra with \SN \geq20. The model dependence of our retrieval results emphasizes the importance of developing a community-wide best-practice for atmospheric retrieval studies.$
$  }$
$   \keywords{   Methods: statistical --$
$                Planets and satellites: terrestrial planets --$
$                Planets and satellites: atmospheres$
$                }$
$   \maketitle$
$\n\end{document}}$
$\newcommand{\life}[0]{LIFE}$
$\newcommand{\lifesim}[0]{LIFE\textsc{sim}}$
$\newcommand{\pI}[0]{Paper~I}$
$\newcommand{\pII}[0]{Paper~II}$
$\newcommand{\pIII}[0]{Paper~III}$
$\newcommand{\pV}[0]{Paper~V}$
$\newcommand{\pIIIaV}[0]{Papers~III and~V}$</div>



<div id="title">

# Large Interferometer For Exoplanets (LIFE):

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.04727-b31b1b.svg)](https://arxiv.org/abs/2303.04727)<mark>Appeared on: 2023-03-09</mark> - _Re-submitted to A&A after a first iteration with referee; 15 pages (main text incl. 8 figures and 6 tables) + appendix; comments are welcome. This paper is part of a series on the LIFE telescope. Related series papers: arXiv:2101.07500, arXiv:2203.00471, arXiv:2112.02054, arXiv:2204.10041_

</div>
<div id="authors">

B. S. Konrad, et al. -- incl., <mark><mark>P. Mollière</mark></mark>

</div>
<div id="abstract">

**Abstract:** Terrestrial exoplanets in the habitable zone are likely a common occurrence. The long-term goal is to characterize the atmospheres of dozens of such objects. The Large Interferometer For Exoplanets ( $\life$ ) initiative aims to develop a spacebased mid-infrared (MIR) nulling interferometer to measure the thermal emission spectra of such exoplanets. We investigate how well $\life$ could characterize a cloudy Venus-twin exoplanet. This allows us to: (1) test our atmospheric retrieval routine on a realistic non-Earth-like MIR emission spectrum of a known planet, (2) investigate how clouds impact retrievals, (3) further refine the $\life$ requirements derived in previous Earth-centered studies. We run Bayesian atmospheric retrievals for simulated $\life$ observations of a Venus-twin exoplanet. Our exoplanet orbits a Sun-like star located $10$ pc from the observer. The $\lifesim$ noise model accounts for all major astrophysical noise sources. We run retrievals using different models (cloudy and cloud-free) and analyze the performance as a function of the quality of the $\life$ observation. This allows us to determine how well atmosphere and clouds are characterizable depending on the quality of the spectrum. At the current minimal resolution ( $\Rv{50}$ ) and signal-to-noise ( $\SNv{10}$ at $\mic{11.2}$ ) requirements for $\life$ , all tested models suggest a $\ce{CO2}$ -rich atmosphere ( $\geq30\%$ in mass fraction). Further, we successfully constrain the atmospheric pressure-temperature ( $\pt$ ) structure above the cloud deck ( $\pt$ uncertainty $\leq\pm15$ K). However, we struggle to infer the main cloud properties. Further, the retrieved planetary radius ( $\Rpl$ ), equilibrium temperature ( $\Teq$ ), and Bond albedo ( $\Ab$ ) depend on the model. Generally, a cloud-free model performs best at the current minimal quality and accurately estimates $\Rpl$ , $\Teq$ , and $\Ab$ . If we consider higher quality spectra (especially $\SNv{20}$ ), we can infer the presence of clouds and pose first constraints on their structure. Our study shows that the minimal $\R$ and $\SN$ requirements for $\life$ suffice to characterize the structure and composition of a Venus-like atmosphere above the cloud deck if an adequate model is chosen. Crucially, the cloud-free model is preferred by the retreival for low spectral qualities. We thus find no direct evidence for clouds at the minimal $\R$ and $\SN$ requirements and cannot infer the thickness of the atmosphere. Clouds are only constrainable in MIR retrievals of spectra with $\SN \geq20$ . The model dependence of our retrieval results emphasizes the importance of developing a community-wide best-practice for atmospheric retrieval studies.

</div>

<div id="div_fig1">

<img src="tmp_2303.04727/./Figures/Validation/val_corner.png" alt="Fig8.1" width="33%"/><img src="tmp_2303.04727/./Figures/Validation/val_pt_structure_residual.png" alt="Fig8.2" width="33%"/><img src="tmp_2303.04727/./Figures/Validation/val_spectrum_residual.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** {Results from the validation retrieval on the $\mic${3-20}, $\R$v{1000}, and photon noise $\SN$v{50} Venus-twin spectrum (opaque \ce{H2SO4} clouds) using the opaque \ce{H2SO4} cloud forward model. *Top left*: Corner plot of the posterior distribution of the forward model parameters. Here, $L(\cdot)$ stands for $\lgrt${\cdot}. We derived the equilibrium temperature $\Teq$  and the Bond albedo $\Ab$  from the other posteriors (see Appendix \ref{sec:albedo_calc}). The dotted black lines indicate the true values. In the 1D posteriors, we show the 16th, 50th, and 84th percentiles as black dashed lines. *Top right*: $\pt$  profile residuals relative to the retrieved median $\pt$  profile. Color-shaded regions indicate percentiles of the retrieved $\pt$  profiles. The gray shaded regions indicate percentiles of the retrieved cloud-top pressure. The solid black line, the orange circular marker, and the red square marker represent the true Venus-twin $\pt$  profile relative to the median retrieved $\pt$  profile. In the inset figure, we plot the 2D $\Ps$ -$\Ts$  posterior. *Bottom*: Residuals of the spectra corresponding to the retrieved posteriors relative to the Venus-twin input. Color-shaded areas represent different quantiles of the residuals. The gray area marks the 1$\sigma$ photon noise-level.} (*fig:valid*)

</div>
<div id="div_fig2">

<img src="tmp_2303.04727/./Figures/R50_SN10/Corner_O_H2SO4.png" alt="Fig10.1" width="25%"/><img src="tmp_2303.04727/./Figures/R50_SN10/PT_O_H2SO4.png" alt="Fig10.2" width="25%"/><img src="tmp_2303.04727/./Figures/R50_SN10/Spec_O_H2SO4.png" alt="Fig10.3" width="25%"/><img src="tmp_2303.04727/./Figures/R50_SN10/EmCont_O_H2SO4.png" alt="Fig10.4" width="25%"/>

**Figure 10. -** Results from the retrieval on the $\mic${4-18.5}, $\R$v{50}, and $\life$sim  noise $\SN$v{10} Venus-twin spectrum (opaque \ce{H2SO4} clouds) using the opaque \ce{H2SO4} cloud forward model (see Sect. \ref{sec:LIFEsim_papIII_ret_res}). *Lower left half*: Corner plot of the posterior distribution of the forward model parameters. Here, $L(\cdot)$ stands for $\lgrt${\cdot}. We derived the equilibrium temperature $\Teq$  and the Bond albedo $\Ab$  from the other posteriors (see Appendix \ref{sec:albedo_calc}). The dotted black lines indicate the true values. In the 1D posteriors, we show the 16th, 50th, and 84th percentiles as black dashed lines. *Top center*: $\pt$  profiles corresponding to the retrieved $\pt$  parameters. Color-shaded regions indicate percentiles of the retrieved $\pt$  profiles. The gray shaded regions indicate percentiles of the retrieved cloud-top pressure. The solid black line, the orange circular marker, and the red square marker represent the true Venus-twin $\pt$  profile. In the inset figure, we plot the 2D $\Ps$ -$\Ts$  posterior (if retrieved; otherwise $\Pct$ -$\Tct$). *Top right*: A table summarizing the true values of the forward model parameters and the 16th, 50th, and 84th percentiles of the parameter posteriors. *Center right*: Spectra corresponding to the retrieved posteriors in comparison to the Venus-twin input. Color-shaded areas represent different quantiles of the retrieved spectra. The solid black line represents the Venus-twin input spectrum. The gray area marks the 1$\sigma$\lifesim  noise-level. Below the spectrum plot, we show the mean wavelength-dependent contribution of the atmospheric layers to the emission spectrum corresponding to the retrieved parameter posteriors. Darker colors indicate higher contributions. The dashed black line indicates the position of the cloud-deck assumed to simulate the input spectrum (opaque \ce{H2SO4} clouds). (*fig:corner_O_H2SO4*)

</div>
<div id="div_fig3">

<img src="tmp_2303.04727/./Figures/Venus_Cloud_vs_Icy.png" alt="Fig1" width="100%"/>

**Figure 1. -** Flux of a cloudy Venus-twin exoplanet (opaque \ce{H2SO4} clouds; dashed green line) compared to an icy world with a thin \ce{CO2}-dominated atmosphere (solid brown line). In the top panel, we plot the UV/O/IR wavelength range. The hatched area represents the expected wavelength-dependent $\SN$v{10} noise for the LUVOIR A mission concept \citep{2017AAS...22940504P}. The thick, solid black line represents the expected $\SN$v{10} noise for the HabEx + Starshade mission concept \citep{2020arXiv200106683G}. In the bottom panel, we plot the MIR wavelength range. The gray shaded region indicates the 1$\sigma$\lifesim  noise level at $\SN$v{10}. (*fig:Clouds_VS_Ice_absolute*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

194  publications in the last 7 days.
	 _build/html/2303.04727.md
	 _build/html/2303.04067.md
	 _build/html/2303.03420.md
	 _build/html/2303.02816.md
	 _build/html/2303.01528.md
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
print(carousel, docs, slides)
print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.
  <div class="carousel" 
       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">
    <div class="carousel-cell"> <div id="slide1" class="md_view">Content 1</div> </div>
  </div> "2303.04727.md" "slide1"
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <!-- Remove caching as much as possible -->
    <meta http-equiv="cache-control" content="no-cache" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Expires" content="-1" />
    <!-- flickity bootstrap CSS -->
    <link rel="stylesheet" href="https://unpkg.com/flickity@2/dist/flickity.min.css">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
        integrity="sha384-EVSTQN3/azprG1Anm